In [ ]:
import numpy as np
from scipy import signal
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
import seaborn as sns
import copy
import random
import pandas as pd
import tqdm
from tqdm import notebook
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F

import scipy.io
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    recall_score,
    f1_score,
    ConfusionMatrixDisplay,
)
import matplotlib.pyplot as plt

torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(0)

# Deep Neural Network (DNN) SSVEP

In [ ]:
class SSVEPDNN(nn.Module):
    def __init__(self, num_classes=40, channels=9, samples=250, subbands=3):
        super(SSVEPDNN, self).__init__()
        # [batch, subbands, channels, time]
        # Subband combination layer
        self.subband_combination = nn.Conv2d(
            subbands, 1, kernel_size=(1, 1), bias=False
        )
        # Channel combination layer
        self.channel_combination = nn.Conv2d(1, 120, kernel_size=(channels, 1))
        # First dropout
        self.drop1 = nn.Dropout(0.1)
        # Third layer - Time convolution
        self.third_conv = nn.Conv2d(120, 120, kernel_size=(1, 2), stride=(1, 2))
        # Second droput
        self.drop2 = nn.Dropout(0.1)
        self.relu = nn.ReLU()
        # 4th conv - FIR filtering
        self.fourth_conv = nn.Conv2d(120, 120, kernel_size=(1, 10), padding="same")
        self.drop3 = nn.Dropout(0.95)

        # Fully connected layer - Classifier
        self.fc = nn.Linear(120 * (samples // 2), num_classes)

        self._initialize_weights()

    def _initialize_weights(self):
        with torch.no_grad():
            self.subband_combination.weight.fill_(1.0)

    def forward(self, x):
        # x shape: [batch, subbands, channels, time]
        x = self.subband_combination(x)  # [batch, 1, channels, time]
        x = self.channel_combination(x)  # [batch, 120, 1, time]
        x = self.drop1(x)
        x = self.third_conv(x)  # [batch, 120, 1, time/2]
        x = self.drop2(x)
        x = self.relu(x)
        x = self.fourth_conv(x)  # [batch, 120, 1, time/2]
        x = self.drop3(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)  # [batch, num_classes]
        output = F.softmax(x, dim=1)
        return output

# Utility Functions and Preprocessing

In [ ]:
def plot_learning_curves(train_losses, val_losses, train_accuracies, val_accuracies):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Perda Treinamento")
    plt.plot(epochs, val_losses, label="Perda Validação")
    plt.xlabel("Épocas")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Curva de Perda: Treino e validação")
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Acurácia Treinamento")
    plt.plot(epochs, val_accuracies, label="Acurácia Validação")
    plt.xlabel("Épocas")
    plt.ylabel("Acurácia (%)")
    plt.legend()
    plt.title("Curva de Acurácia: Treino e validação")
    plt.tight_layout()
    plt.show()

In [ ]:
all_data = np.array(all_data)
all_data.shape

In [ ]:
def filter_signals_subbands(eeg_signals, subban_no, sampling_rate):
    samples, total_channels, sample_length = eeg_signals.shape

    AllData = np.zeros((samples, subban_no, total_channels, sample_length))

    # Bandpass filters
    high_cutoff = [90] * subban_no
    low_cutoff = [i for i in range(8, 8 * (subban_no + 1), 8)]
    filter_order = 2
    passband_ripple = 1
    bp_filters = []

    for i in range(subban_no):
        b, a = signal.cheby1(
            filter_order,
            passband_ripple,
            [low_cutoff[i], high_cutoff[i]],
            btype="band",
            fs=sampling_rate,
        )
        bp_filters.append((b, a))

    # Filtering
    for sample in range(samples):
        tmp_raw = eeg_signals[sample]
        for sub_band in range(subban_no):
            processed_signal = np.zeros((total_channels, sample_length))
            b, a = bp_filters[sub_band]

            for ch_idx in range(total_channels):
                processed_signal[ch_idx] = scipy.signal.filtfilt(b, a, tmp_raw[ch_idx])

            AllData[sample, sub_band, :, :] = processed_signal
    print(f"All data shape after filtering: {AllData.shape}")
    return AllData

In [ ]:
def separar_em_janelas(matriz, tamanho_janela, incluir_ultimo=False):
    total_linhas = matriz.shape[0]
    num_janelas = total_linhas // tamanho_janela
    janelas = []
    for i in range(0, total_linhas, tamanho_janela):
        janela = matriz[i : i + tamanho_janela]
        janelas.append(janela)
    if not incluir_ultimo and total_linhas % tamanho_janela != 0:
        janelas.pop()
    return janelas, num_janelas

In [ ]:
def train(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    num_epochs=100,
    device=0,
    save_path="best_model.pth",
):
    best_val_accuracy = -float("inf")
    model.to(device)
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    for epoch in tqdm.notebook.tqdm(range(num_epochs)):
        model.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)
        train_accuracy = train_correct / train_total
        avg_train_loss = running_loss / len(train_loader)
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.inference_mode():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)
        val_accuracy = val_correct / val_total
        avg_val_loss = val_loss / len(val_loader)
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), save_path)
        print(
            f"Epoch {epoch + 1}/{num_epochs}: "
            f"Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
            f"Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}"
        )
    plot_learning_curves(train_losses, val_losses, train_accuracies, val_accuracies)
    model.load_state_dict(best_model)
    return model

In [ ]:
def evaluate(model, test_loader, chanels):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.inference_mode():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds, average="weighted")
    f1 = f1_score(all_labels, all_preds, average="weighted")
    cm = confusion_matrix(all_labels, all_preds)
    print(f"Test set Accuracy: {accuracy:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    classes = np.unique(np.concatenate((all_labels, all_preds)))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    fig, ax = plt.subplots(figsize=(15, 15))
    disp.plot(ax=ax, cmap="Blues", xticks_rotation="vertical")
    plt.show()
    return accuracy, recall, f1, cm

In [ ]:
frequencias_e_fases = scipy.io.loadmat(
    "C:/Users/machi/Documents/Mestrado/repos/data/benchmark/Freq_Phase.mat"
)
frequencias = frequencias_e_fases["freqs"]
frequencias = np.round(frequencias, 2).ravel()
fases = frequencias_e_fases["phases"]

sample_rate = 250
delay = 160
tamanho_da_janela = 1
tamanho_da_janela = int(np.ceil(tamanho_da_janela * sample_rate))
occipital_electrodes = np.array([47, 53, 54, 55, 56, 57, 60, 61, 62])
frequencias_desejadas = frequencias[:]
indices = [np.where(frequencias == freq)[0][0] for freq in frequencias_desejadas]
epochs = 1000
exp_dir = Path("dnn_10_freqs_04s_cm")
seed = 42
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)

In [ ]:
print("Frequências de interesse:", frequencias_desejadas)
print("Indices das frequências de interesse:", indices)

In [ ]:
def load_data_from_all_users():
    all_data = []
    for user in tqdm.notebook.tqdm(range(1, 11), desc="Carregando dados dos usuários"):
        file_path = (
            f"C:/Users/machi/Documents/Mestrado/repos/data/benchmark/S{user}.mat"
        )
        data = scipy.io.loadmat(file_path)["data"]
        data = data[:, (delay) : (delay + 1250), :, :]
        all_data.append(data)
    return all_data

In [ ]:
all_data = load_data_from_all_users()

In [ ]:
all_data[0].shape

# Leave-One-User-Out Cross Validation


In [ ]:
metricas_usuarios = []
exp_dir.mkdir(parents=True, exist_ok=True)

for user in range(1, len(all_data) + 1):
    print(f"Processando Usuário {user}")
    n_freqs_sel = len(indices)
    metricas_crossval = []
    users_train = [u for u in range(1, len(all_data) + 1) if u != user]
    user_test = user

    x_train = []
    labels_train = []
    x_test = []
    labels_test = []

    for u in users_train:
        data = all_data[u - 1]
        for session in range(data.shape[3]):
            for freq in range(len(indices)):
                eeg_trial = data[occipital_electrodes, :, indices[freq], session]
                eeg_trial = eeg_trial[:, :tamanho_da_janela]
                # eeg_trial_janelas, numero_janelas = separar_em_janelas(
                #     eeg_trial.T, tamanho_da_janela
                # )
                # eeg_trial_janelas_array = np.stack(eeg_trial_janelas)
                # eeg_trial_janelas_array = eeg_trial_janelas_array.transpose(0, 2, 1)
                # x_train.append(eeg_trial_janelas_array)
                # labels_train.extend([frequencias[freq]] * numero_janelas)
                x_train.append(eeg_trial)
                labels_train.extend([frequencias[freq]])

    # x_train = np.concatenate(x_train, axis=0)
    x_train = np.array(x_train)
    x_train = filter_signals_subbands(x_train, subban_no=3, sampling_rate=250)
    # Test user
    data = all_data[user_test - 1]
    for session in range(data.shape[3]):
        for freq in range(len(indices)):
            eeg_trial = data[occipital_electrodes, :, indices[freq], session]
            eeg_trial = eeg_trial[:, :tamanho_da_janela]
            # eeg_trial_janelas, numero_janelas = separar_em_janelas(
            #     eeg_trial.T, tamanho_da_janela
            # )
            # eeg_trial_janelas_array = np.stack(eeg_trial_janelas)
            # eeg_trial_janelas_array = eeg_trial_janelas_array.transpose(0, 2, 1)
            # x_test.append(eeg_trial_janelas_array)
            # labels_test.extend([frequencias[freq]] * numero_janelas)
            x_test.append(eeg_trial)
            labels_test.extend([frequencias[freq]])

    # x_test = np.concatenate(x_test, axis=0)
    x_test = np.array(x_test)
    x_test = filter_signals_subbands(x_test, subban_no=3, sampling_rate=250)

    mapeamento = {rotulo: i for i, rotulo in enumerate(sorted(frequencias_desejadas))}

    # Labels
    rotulos_treinamento = torch.tensor(
        [mapeamento[rotulo.item()] for rotulo in labels_train]
    )
    labels_test = torch.tensor([mapeamento[rotulo.item()] for rotulo in labels_test])

    x_train = torch.from_numpy(x_train.copy()).float().to(device)
    X_test = torch.from_numpy(x_test.copy()).float().to(device)
    Y_treino = rotulos_treinamento.to(torch.long).to(device)
    Y_teste = labels_test.to(torch.long).to(device)
    print("x_train:", x_train.shape)
    print("X_test:", X_test.shape)
    print("Y_treino:", Y_treino.shape)
    print("Y_teste:", Y_teste.shape)
    model = SSVEPDNN(
        num_classes=len(frequencias_desejadas),
        channels=9,
        samples=tamanho_da_janela,
        subbands=3,
    ).to(device)
    dataset = TensorDataset(x_train, Y_treino)
    train_size = int(0.85 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(
        dataset,
        [train_size, val_size],
        generator=torch.Generator().manual_seed(seed),
    )
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    test_loader = DataLoader(
        TensorDataset(X_test, Y_teste), batch_size=10, shuffle=False
    )
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    best_model = train(
        model,
        train_loader,
        val_loader,
        criterion,
        optimizer,
        num_epochs=epochs,
        device=device,
        save_path=exp_dir.joinpath(f"best_model_user_{user}.pth"),
    )
    accuracy, recall, f1, cm = evaluate(
        best_model, test_loader, chanels=len(frequencias_desejadas)
    )
    metricas_crossval.append(
        {
            "usuario": user,
            "acuracia": accuracy,
            "recall": recall,
            "f1-score": f1,
            "confusion_matrix": cm,
        }
    )
    print(
        f"Usuário {user} Finalizado: Acurácia={accuracy:.4f}, Recall={recall:.4f}, F1={f1:.4f}"
    )
    metricas_usuarios.extend(metricas_crossval)
    df_metricas = pd.DataFrame(metricas_usuarios)
    df_metricas.to_csv(exp_dir.joinpath(f"{exp_dir}_metricas.csv"), index=False)
    print("-" * 50)

# Leave-One-Session-Out Cross Validation (General + Fine-tune)

In [ ]:
frequencias_e_fases = scipy.io.loadmat(
    "C:/Users/machi/Documents/Mestrado/repos/data/benchmark/Freq_Phase.mat"
)
frequencias = frequencias_e_fases["freqs"]
frequencias = np.round(frequencias, 2).ravel()
fases = frequencias_e_fases["phases"]


# Parâmetros de pré-processamento
sample_rate = 250
delay = 160  # 160 amostras, 0,5s (sem estimulação) + 0,14s (latencia para começo da evocação)

# Parâmetros de janelas e sessões
tamanho_da_janela = 1
tamanho_da_janela = int(np.ceil(tamanho_da_janela * sample_rate))

# Eletrodos e frequências de interesse
occipital_electrodes = np.array([47, 53, 54, 55, 56, 57, 60, 61, 62])
frequencias_desejadas = frequencias[:]
indices = [np.where(frequencias == freq)[0][0] for freq in frequencias_desejadas]

epochs_general = 100
epochs_fine_tune = 25


exp_dir = Path("dnn_leave-one-session-out-10_freqs_1s")

In [ ]:
exp_dir.mkdir(parents=True, exist_ok=True)
general_models_dir = exp_dir.joinpath("general_models")
general_models_dir.mkdir(parents=True, exist_ok=True)
fine_tuned_models_dir = exp_dir.joinpath("fine_tuned_models")
fine_tuned_models_dir.mkdir(parents=True, exist_ok=True)
metricas_usuarios = []
num_sessions = 6
num_users = len(all_data)

for sessao_teste in range(num_sessions):
    print(f"Sessão de teste: {sessao_teste}")
    x_train_general = []
    y_train_general = []

    for user in range(num_users):
        data = all_data[user]
        for sessao in range(num_sessions):
            if sessao == sessao_teste:
                continue
            for freq in range(len(indices)):
                eeg_trial = data[occipital_electrodes, :, indices[freq], sessao]
                eeg_trial = eeg_trial[:, :tamanho_da_janela]
                x_train_general.append(eeg_trial)
                y_train_general.extend([frequencias[freq]])
                # eeg_trial_janelas, numero_janelas = separar_em_janelas(
                #     eeg_trial.T, tamanho_da_janela
                # )
                # eeg_trial_janelas_array = np.stack(eeg_trial_janelas).transpose(0, 2, 1)
                # x_train_general.append(eeg_trial_janelas_array)
                # y_train_general.extend([frequencias[freq]] * numero_janelas)
    x_train_general = np.array(x_train_general)
    mapeamento = {rotulo: i for i, rotulo in enumerate(sorted(frequencias_desejadas))}
    y_train_general_tensor = torch.tensor(
        [mapeamento[rotulo.item()] for rotulo in y_train_general]
    )
    # DNN expects [batch, subbands, channels, time]
    x_train_general = filter_signals_subbands(
        x_train_general, subban_no=3, sampling_rate=250
    )
    X_train_general = torch.from_numpy(x_train_general.copy()).float().to(device)
    Y_train_general = y_train_general_tensor.to(torch.long).to(device)
    model_general = SSVEPDNN(
        num_classes=len(frequencias_desejadas),
        channels=9,
        samples=tamanho_da_janela,
        subbands=3,
    ).to(device)
    dataset_general = TensorDataset(X_train_general, Y_train_general)
    train_size = int(0.85 * len(dataset_general))
    val_size = len(dataset_general) - train_size
    train_dataset, val_dataset = random_split(
        dataset_general,
        [train_size, val_size],
        generator=torch.Generator().manual_seed(seed),
    )
    train_loader = DataLoader(dataset_general, batch_size=64, shuffle=True)
    val_loader = DataLoader(dataset_general, batch_size=16, shuffle=False)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model_general.parameters(), lr=0.0001)
    print("Treinando modelo geral...")
    best_model_general = train(
        model_general,
        train_loader,
        val_loader,
        criterion,
        optimizer,
        num_epochs=epochs_general,
        device=device,
        save_path=general_models_dir.joinpath(
            f"best_general_model_session_{sessao_teste}.pth"
        ),
    )

    # Fine-tuning para cada usuário
    for user in range(num_users):
        print(f"Usuário {user+1} - Fine-tuning e validação na sessão {sessao_teste}")
        data = all_data[user]
        x_finetune = []
        y_finetune = []
        for sessao in range(num_sessions):
            if sessao == sessao_teste:
                continue
            for freq in range(len(indices)):
                eeg_trial = data[occipital_electrodes, :, indices[freq], sessao]
                eeg_trial = eeg_trial[:, :tamanho_da_janela]
                x_finetune.append(eeg_trial)
                y_finetune.extend([frequencias[freq]])
                # eeg_trial_janelas, numero_janelas = separar_em_janelas(
                #     eeg_trial.T, tamanho_da_janela
                # )
                # eeg_trial_janelas_array = np.stack(eeg_trial_janelas).transpose(0, 2, 1)
                # x_finetune.append(eeg_trial_janelas_array)
                # y_finetune.extend([frequencias[freq]] * numero_janelas)
        x_finetune = np.array(x_finetune)
        x_finetune = filter_signals_subbands(x_finetune, subban_no=3, sampling_rate=250)
        X_finetune = torch.from_numpy(x_finetune.copy()).float().to(device)
        y_finetune_tensor = torch.tensor(
            [mapeamento[rotulo.item()] for rotulo in y_finetune]
        )
        Y_finetune = y_finetune_tensor.to(torch.long).to(device)
        x_test = []
        y_test = []
        for freq in range(len(indices)):
            eeg_trial = data[occipital_electrodes, :, indices[freq], sessao_teste]
            eeg_trial = eeg_trial[:, :tamanho_da_janela]
            x_test.append(eeg_trial)
            y_test.extend([frequencias[freq]])
            # eeg_trial_janelas, numero_janelas = separar_em_janelas(
            #     eeg_trial.T, tamanho_da_janela
            # )
            # eeg_trial_janelas_array = np.stack(eeg_trial_janelas).transpose(0, 2, 1)
            # x_test.append(eeg_trial_janelas_array)
            # y_test.extend([frequencias[freq]] * numero_janelas)

        x_test = np.array(x_test)
        x_test = filter_signals_subbands(x_test, subban_no=3, sampling_rate=250)
        X_test = torch.from_numpy(x_test.copy()).float().to(device)
        y_test_tensor = torch.tensor([mapeamento[rotulo.item()] for rotulo in y_test])
        Y_test = y_test_tensor.to(torch.long).to(device)
        model_finetune = SSVEPDNN(
            num_classes=len(frequencias_desejadas),
            channels=9,
            samples=tamanho_da_janela,
            subbands=3,
        ).to(device)
        model_finetune.load_state_dict(best_model_general.state_dict())
        dataset_finetune = TensorDataset(X_finetune, Y_finetune)
        train_size = int(0.85 * len(dataset_finetune))
        val_size = len(dataset_finetune) - train_size
        train_dataset, val_dataset = random_split(
            dataset_finetune,
            [train_size, val_size],
            generator=torch.Generator().manual_seed(seed),
        )
        train_loader = DataLoader(dataset_finetune, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
        test_loader = DataLoader(
            TensorDataset(X_test, Y_test), batch_size=10, shuffle=False
        )
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model_finetune.parameters(), lr=0.00005)
        print("Fine-tuning modelo para o usuário...")
        best_model_finetune = train(
            model_finetune,
            train_loader,
            test_loader,
            criterion,
            optimizer,
            num_epochs=epochs_fine_tune,
            device=device,
        )
        accuracy, recall, f1, cm = evaluate(
            best_model_finetune, test_loader, chanels=len(frequencias_desejadas)
        )
        metricas_usuarios.append(
            {
                "usuario": user + 1,
                "sessao_teste": sessao_teste,
                "acuracia": accuracy,
                "recall": recall,
                "f1-score": f1,
                "confusion_matrix": cm,
            }
        )
        print(
            f"Usuário {user+1} - Sessão {sessao_teste} Finalizada: Acurácia={accuracy:.4f}, Recall={recall:.4f}, F1={f1:.4f}"
        )
    print("-" * 50)
    df_metricas = pd.DataFrame(metricas_usuarios)
    df_metricas.to_csv(
        exp_dir.joinpath(f"{exp_dir}_metricas_leave_one_session_out_finetune.csv"),
        index=False,
    )